In [46]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta

symbol = "NVDA.NE"
period = '1y'
interval = '1h'

# 1 day = 7 hours from 14:30 - 20:30
df = yf.download(tickers=symbol, period=period, interval=interval)
df = df.reset_index()
df.columns = df.columns.get_level_values(0)


df.to_csv("NVDA.csv")
df.head()
# df["Close"]

[*********************100%***********************]  1 of 1 completed


Price,Datetime,Close,High,Low,Open,Volume
0,2024-01-04 14:30:00+00:00,11.3675,11.3900,11.2750,11.3125,0
1,2024-01-04 15:30:00+00:00,11.4725,11.4725,11.3700,11.3750,11550
2,2024-01-04 16:30:00+00:00,11.4450,11.4900,11.4400,11.4550,8963
3,2024-01-04 17:30:00+00:00,11.4400,11.4400,11.4000,11.4325,4662
4,2024-01-04 18:30:00+00:00,11.4175,11.4600,11.4175,11.4500,2916


In [47]:
import pandas_ta as ta
from scipy.signal import argrelextrema
import numpy as np



df['200EMA'] = ta.ema(df['Close'], length=200) # intermediate
ped_df = (df['Close'] - df['200EMA']) / df['Close']
df['Price_200EMA_diff'] = ped_df

adx = ta.adx(high=df['High'], low=df['Low'], close=df['Close'], length=14)
df['ADX'] = adx['ADX_14'] # intermediate

df['Short_MA'] = df['Close'].rolling(window=20).mean() # intermediate
df['Long_MA'] = df['Close'].rolling(window=50).mean() # intermediate

df['MA_CO_signal'] = 0
df.loc[(df['Short_MA'] > df['Long_MA']) & (df['ADX'] > 25), 'MA_CO_signal'] = 1  
df.loc[(df['Short_MA'] <= df['Long_MA']) & (df['ADX'] > 25), 'MA_CO_signal'] = -1  



# order = 5  
# df['Swing_High'] = df.iloc[argrelextrema(df['High'].values, np.greater, order=order)[0]]['High'] # intermediate
# df['Swing_Low'] = df.iloc[argrelextrema(df['Low'].values, np.less, order=order)[0]]['Low'] # intermediate
# df['Current_Support'] = np.nan # intermediate
# df['Current_Resistance'] = np.nan # intermediate
# current_support = np.nan
# current_resistance = np.nan
# tolerance = 0.01

# for idx, row in df.iterrows():
#     if not np.isnan(row['Swing_Low']):
#         current_support = row['Swing_Low']
#     if not np.isnan(row['Swing_High']):
#         current_resistance = row['Swing_High']
    
#     df.at[idx, 'Current_Support'] = current_support
#     df.at[idx, 'Current_Resistance'] = current_resistance

# df['Close_Support_Diff'] = (df['Close'] - df['Current_Support']) / df['Current_Support']  # intermediate
# df['Close_Resistance_Diff'] = (df['Close'] - df['Current_Resistance']) / df['Current_Resistance']  # intermediate

# df['On_Support'] = df['Close_Support_Diff'].abs() <= tolerance # intermediate
# df['On_Resistance'] = df['Close_Resistance_Diff'].abs() <= tolerance # intermediate 

# df['Support_Lag1'] = df['Current_Support'].shift(1)
# df['Resistance_Lag1'] = df['Current_Resistance'].shift(1)
# df['Distance_to_Support'] = df['Close_Support_Diff']
# df['Distance_to_Resistance'] = df['Close_Resistance_Diff']

# df.drop(['Current_Support'])
# df.drop(['Current_Resistance'])

df


Price,Datetime,Close,High,Low,Open,Volume,200EMA,Price_200EMA_diff,ADX,Short_MA,Long_MA,MA_CO_signal
0,2024-01-04 14:30:00+00:00,11.367500,11.390000,11.275000,11.312500,0,NaN,NaN,NaN,NaN,NaN,0
1,2024-01-04 15:30:00+00:00,11.472500,11.472500,11.370000,11.375000,11550,NaN,NaN,NaN,NaN,NaN,0
2,2024-01-04 16:30:00+00:00,11.445000,11.490000,11.440000,11.455000,8963,NaN,NaN,NaN,NaN,NaN,0
3,2024-01-04 17:30:00+00:00,11.440000,11.440000,11.400000,11.432500,4662,NaN,NaN,NaN,NaN,NaN,0
4,2024-01-04 18:30:00+00:00,11.417500,11.460000,11.417500,11.450000,2916,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1756,2025-01-03 16:30:00+00:00,33.529999,33.619999,33.419998,33.490002,182331,32.106672,0.042449,20.510243,32.1365,32.0264,0
1757,2025-01-03 17:30:00+00:00,33.720001,33.770000,33.520000,33.549999,163964,32.122725,0.047369,22.235394,32.2000,32.0914,0
1758,2025-01-03 18:30:00+00:00,33.689999,33.820000,33.560001,33.709999,191476,32.138320,0.046058,23.903770,32.2670,32.1466,0
1759,2025-01-03 19:30:00+00:00,33.660000,33.720001,33.599998,33.669998,90993,32.153461,0.044758,25.452976,32.3430,32.1946,1


In [48]:
# Count rows with at least one NaN
num_rows_with_nan = df.isna().any(axis=1).sum()

print(f"Number of rows with at least one NaN: {num_rows_with_nan}")

df.shape

Number of rows with at least one NaN: 199


(1761, 12)

In [49]:
df.dropna(inplace=True)
df = df.iloc[4:] # now data starts from 2024-02-15, and each day has 7 rows from 14:30 - 20:30 inclusive
df.shape


(1558, 12)

In [50]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
ohe_features = ohe.fit_transform(df[['MA_CO_signal']])
ohe_df = pd.DataFrame(ohe_features, columns=ohe.get_feature_names_out(['MA_CO_signal']))
# ohe_df.head(30)
# df_encoded = pd.concat([df, ohe_df], axis = 1)

# df_encoded.head(30)
# Reset indices
df_reset = df.reset_index(drop=True)
ohe_df_reset = ohe_df.reset_index(drop=True)

# Concatenate
df_encoded = pd.concat([df_reset, ohe_df_reset], axis=1)
df = df_encoded


/Users/guanyulu/anaconda3/envs/btcp/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [51]:
def add_future_price(df, shift_days=1):
    df['Next Close'] = df['Close'].shift(-shift_days)
    df['T_reg'] = (df['Next Close'] - df['Close']) / df['Close'] * 100
    df['T_cla'] = df['T_reg'].apply(lambda x: 1 if x > 0 else 0)
    return df

df = add_future_price(df)
df = df.dropna()

In [52]:
df.to_csv("NVDA_cleaned.csv")
df.shape

(1557, 18)